In [97]:
import pip
pip.main(["install", "google.cloud"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting google.cloud

Using cached google_cloud-0.34.0-py2.py3-none-any.whl (1.8 kB)

Installing collected packages: google.cloud

Successfully installed google.cloud-0.34.0

WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv


0

In [98]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt 
import datetime
import time
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [99]:
data = pd.read_csv('gs://fraud-raw-data/bank_transactions.csv')

In [100]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [101]:
data.shape

(1048567, 9)

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [103]:
data.isna().sum()

TransactionID                 0
CustomerID                    0
CustomerDOB                3397
CustGender                 1100
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [104]:
data.dropna(axis=0,inplace = True)

In [105]:
data['CustomerDOB'] = pd.to_datetime(data['CustomerDOB'], dayfirst=True)

In [106]:
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'], dayfirst=True)

In [107]:
def fix_date(x):
    if (x.year > 2023) or (x.year < 1972):
        year = x.year - 100
    else:
        year = x.year
    return datetime.date(year,x.month,x.day)

In [108]:
data['CustomerDOB'] = data['CustomerDOB'].apply(fix_date)
data['TransactionDate'] = data['TransactionDate'].apply(fix_date)

In [109]:
data['Age']=(data['TransactionDate'].values-data['CustomerDOB'].values)/365

In [110]:
def years(x):
    return x.days

In [111]:
data['Age']=data['Age'].apply(years)

In [112]:
numeric = ['CustAccountBalance','TransactionAmount (INR)','Age']
datatime = ['CustomerDOB','TransactionDate','TransactionTime']
categorical = ['CustGender','CustomerID','CustLocation']

In [113]:
data[data['Age']  >200].shape

(56292, 10)

In [114]:
data = data[data["CustGender"]!='T']

In [115]:
week_day = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

In [116]:
data['TransactionDay'] = data['TransactionTime'].apply(time.ctime)
data['tr_year']        = data['TransactionDate'].apply(lambda x : x.year)
data['tr_month']       = data['TransactionDate'].apply(lambda x : x.month)
data['tr_day']         = data['TransactionDate'].apply(lambda x : x.day)
data['tr_weekday']     = data['TransactionDate'].apply(lambda x : week_day[pd.Timestamp(x).dayofweek])

In [117]:
hh=[int(datetime.datetime.fromtimestamp(i).strftime("%H")) for i in data.TransactionTime.values]
data['hr BY hr'] = hh

In [118]:
time = []
for i in hh:
    if i >=0 and i < 13:
        time.append("Morning")
    elif i >= 13 and i<=17:
        time.append("Afternoon")
    elif i > 17 and i <= 19:
        time.append("Evening")
    elif i > 19:
        time.append("Night")
data['tr_pr'] = time

In [119]:
data['tr_ratio'] = data['TransactionAmount (INR)'] *100 / (data['CustAccountBalance']+0.01)

In [120]:
le = preprocessing.LabelEncoder()
  
data['CustGender']= le.fit_transform(data['CustGender'])
data['CustLocation']= le.fit_transform(data['CustLocation'])
data['tr_pr']= le.fit_transform(data['tr_pr'])
data['tr_weekday']= le.fit_transform(data['tr_weekday'])
 

In [121]:
data = data.drop(['TransactionID','CustomerID','CustomerDOB','TransactionDate','TransactionTime', 'TransactionDay'],axis=1)

In [122]:
data = data.drop(data[data['TransactionAmount (INR)'] == 0].index,axis = 0)

In [123]:
columns_name =data.columns

In [125]:
s = StandardScaler()
data = s.fit_transform(data)
data = pd.DataFrame(data,columns=columns_name)
data.head()

,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),Age,tr_year,tr_month,tr_day,tr_weekday,hr BY hr,tr_pr,tr_ratio
0,-1.646425,-0.223738,-0.114717,-0.234970,-0.370752,0.0,-0.771889,-1.341525,1.021644,0.702597,-1.802404,-0.017138
1,0.607376,-0.197927,-0.133086,4.026871,0.198221,0.0,-0.771889,-1.341525,1.021644,0.702597,-1.802404,-0.016812
2,-1.646425,0.479512,-0.114652,-0.168850,-0.416885,0.0,-0.771889,-1.341525,1.021644,0.702597,-1.802404,-0.017137
3,-1.646425,0.479512,0.887937,0.075063,-0.063199,0.0,-0.771889,-1.341525,1.021644,0.702597,-1.802404,-0.017138
4,-1.646425,0.643265,-0.127836,0.029739,-0.278486,0.0,-0.771889,-1.341525,1.021644,-1.296590,0.351364,-0.017131


In [126]:
data.to_csv('gs://fraud-processed-data/bank_transactions_processed.csv', index =False)